In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from time import time

import DynamicTimeAllocationModel

path = 'output/'

# c++ settings
do_compile = True
threads = 20

# from EconModel import cpptools
# cpptools.setup_nlopt(folder='cppfuncs/', do_print=True)

# Solve and simulate from alternative models

Benchmark model settings

In [2]:
settings = { 
       'T':20,
       'p_meet': 0.1, 
       'div_A_share': 0.5, 
       'sigma_love':0.1, 

       'num_love': 21, 
       'num_A': 50, 
       'num_A_pd':50,

       'simT':20,
       'simN': 10000,

       'threads':threads,
       'do_egm': True,
       }

Solve application model

In [3]:
model = DynamicTimeAllocationModel.HouseholdModelClass(par=settings)
model.link_to_cpp(force_compile=do_compile)
%time model.solve()

CPU times: total: 3.59 s
Wall time: 552 ms
